# <font style="color:blue"> Squat Checker</font>

In this notebook, lets see how squat checker application can be built using pose estimation model.

Squat checker will perform following 2 checks:
- right squat pose by measuring the angles of knees, hips and ankles
- count the number of squats

<img src='https://www.learnopencv.com/wp-content/uploads/2020/06/c3-w13-squat-1.png'>

We will use detectron2 to load model and run inference. Here, we show the example of single person videos, the same implementation can be extended for squat check on multiple people in the same image by adding display to more than one person.

## <font style="color:green">Setup Code</font>

To use the detectron2's densepose training module, we will setup the detectron2 code.

In [2]:
# install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

'2.6.0+cu124'

In [3]:
!git clone https://github.com/facebookresearch/detectron2 detectron2
!pip install -e detectron2

Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15837 (delta 30), reused 12 (delta 12), pack-reused 15771 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (11532/11532), done.
Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.7/458.7 kB 30.5 MB/s eta 0:00:

## <font style="color:green"> Import Libraries</font>

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import os
import time

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

## <font style="color:green"> Setup Config</font>

Here, we will import detectron2's Keypoint RCNN model for keypoints detection.

- Import default config
- Import model config file and weights file
- Set threshold for the model as 0.5
- Initiate default predictor object with the above config

In [2]:
start = time.time()
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
model_load_done = time.time()
print("model_load", model_load_done - start)

[03/31 11:17:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...


model_final_a6e10b.pkl: 237MB [00:01, 176MB/s]                           


model_load 2.9173295497894287


## <font style="color:green"> Helper functions</font>


Selects person ids whose score is greater than 0.9

In [3]:
def findPersonIndicies(scores):
    return [i for i, s in enumerate(scores) if s > 0.9]

Finds the slope between two points (x1, y1) and (x2, y2)

In [4]:
def findSlope(x1, y1, x2, y2):
    return float(y2-y1)/(x2-x1)

Finds the angle of the line w.r.t +ve X-axis in counterclockwise direction

In [5]:
import math

def findAngle(x1, y1, x2, y2):
    return math.atan2(y1 - y2, x1 - x2)

Finds the angle between two lines of slope m1 and m2

In [6]:
def findAngleBtLines(m1, m2):
    PI = 3.14
    angle = math.atan((m2 -  m1)/(1 + m1*m2))

    return (angle*180)/PI

For the selected persons, collects the required key points among 17 key points
- 11-Left hip-0
- 12-Right hip-1
- 13-Left Knee-2
- 14-Right Knee-3
- 15-Left Ankle-4
- 16-Right Ankle-5

In [7]:
def filterPersons(outputs):
    persons = {}
    pIndicies = findPersonIndicies(outputs["instances"].scores)

    for x in pIndicies:
        desired_kp = outputs["instances"].pred_keypoints[x][11:].to("cpu")
        persons[x] = desired_kp

    return (persons, pIndicies)

In [18]:
#def drawLine(image, P1, P2, color):
#    cv2.line(image, P1, P2, color, thickness=3, lineType=8)

def drawLine(image, P1, P2, color):
    # Convert P1 and P2 to integer tuples
    P1 = (int(P1[0]), int(P1[1]))
    P2 = (int(P2[0]), int(P2[1]))
    cv2.line(image, P1, P2, color, thickness=3, lineType=8)

In [9]:
def putTextOnImage(image, text, X, Y, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2

    cv2.putText(image, text,(X, Y),font, font_scale,color,font_thickness,cv2.LINE_AA)

## <font style="color:green"> Measurement of angles based on key points</font>
* Find the slope of line connecting knee-ankle, hip-knee for the left and right sides and slope of line connecting knees.
* Based on the above slopes find the angle between left knee-ankle and right knee-ankle line. This angle has to be +ve when the person is in a correct squat position.
* The angle between left hip-knee line to the knee-knee line and the right hip-knee line to the knee-knee line should be below 30 degrees.
* We draw the corresponding lines with same color and use the same color to represent the angle between them.


In [10]:
#11-Left hip-0
#12-Right hip-1
#13-Left Knee-2
#14-Right Knee-3
#15-Left Ankle-4
#16-Right Ankle-5
kp_mapping = {"Left Hip": 0, "Right Hip": 1, "Left Knee": 2, "Right Knee": 3, "Left Ankle": 4, "Right Ankle": 5}

def drawKeypoints(outputs, im):
    persons, pIndicies = filterPersons(outputs)
    img = im.copy()

    angles_output = {}

    for i in pIndicies:
        l_arr1 = persons[i][2]
        l_arr2 = persons[i][4]
        l_arr3 = persons[i][0]
        r_arr1 = persons[i][3]
        r_arr2 = persons[i][5]
        r_arr3 = persons[i][1]

        left_ka_slope = findSlope(l_arr1[0], l_arr1[1], l_arr2[0], l_arr2[1])
        left_kh_slope = findSlope(l_arr3[0], l_arr3[1], l_arr1[0], l_arr1[1])
        right_ka_slope = findSlope(r_arr1[0], r_arr1[1], r_arr2[0], r_arr2[1])
        right_kh_slope = findSlope(r_arr3[0], r_arr3[1], r_arr1[0], r_arr1[1])
        kk_slope = findSlope(r_arr1[0], r_arr1[1], l_arr1[0], l_arr1[1])

        angle_btw_knees = findAngleBtLines(right_ka_slope, left_ka_slope)
        left_hk_angle = findAngleBtLines(kk_slope, left_kh_slope)
        right_hk_angle = findAngleBtLines(right_kh_slope, kk_slope)

        angles_output[i] = [right_hk_angle, left_hk_angle, angle_btw_knees]

        #Considering only one person
        if i == 0:
            if not math.isnan(angle_btw_knees):
                knees_ctr_pt = (np.array(l_arr2) + np.array(r_arr2))/2
                putTextOnImage(img, str(int(angle_btw_knees)), int(knees_ctr_pt[0]) - 10, int(knees_ctr_pt[1]),
                               (0,255,0))

            if not math.isnan(left_hk_angle):
                left_hk_pt = l_arr1
                putTextOnImage(img, str(int(left_hk_angle)), int(left_hk_pt[0]) + 10, int(left_hk_pt[1]),
                               (255,255,0))

            if not math.isnan(right_hk_angle):
                right_hk_pt = r_arr1
                putTextOnImage(img, str(int(right_hk_angle)), int(right_hk_pt[0]) - 40, int(right_hk_pt[1]),
                               (255,255,0))

            ##Draw left knee ankle line
            drawLine(img, (l_arr1[0], l_arr1[1]), (l_arr2[0], l_arr2[1]), (0, 255, 0))

            ##Draw left hip knee line
            drawLine(img, (l_arr3[0], l_arr3[1]), (l_arr1[0], l_arr1[1]), (255, 255, 0))

            ##Draw right knee ankle line
            drawLine(img, (r_arr1[0], r_arr1[1]), (r_arr2[0], r_arr2[1]), (0, 255, 0))

            ##Draw right hip knee line
            drawLine(img, (r_arr3[0], r_arr3[1]), (r_arr1[0], r_arr1[1]), (255, 255, 0))

            ##Draw knees connecting and hips connecting line
            drawLine(img, (r_arr1[0], r_arr1[1]), (l_arr1[0], l_arr1[1]), (255, 255, 0))
            drawLine(img, (r_arr3[0], r_arr3[1]), (l_arr3[0], l_arr3[1]), (255, 0, 0))

    return img, angles_output

In [11]:
def predict(im):
    model_start = time.time()
    outputs = predictor(im)
    model_out = time.time()
    # print("model output time", model_out - model_start)
    out, angles_out = drawKeypoints(outputs, im)
    # print("process and draw output", time.time() - model_out)

    return out, angles_out

## <font style="color:green"> Inference on Video</font>

Below function takes the video path as input and returns the output video which shows the squat counter and the respective angles of the lower  body.

- Every alternate frame is processed to reduce the processing time (`n_frame=2`)
- As mentioned in the measurement of angles section, we use the conditions on angles after each frame is processed.
- Squat count is increased whenever the previous frame doesn't meet the conditions and the current frame meets the conditions.
- If any immediate frame after correct squat frame misses the squat condition and gets back to the correct squat frame again, then it is handled to not increase the squat count unless the differences of frames between `2 squats > 10 frames`.

**[Download the Input Video](https://www.dropbox.com/s/ygpbc5q0xtrsjqq/ProperSquatTechnique_cut.mp4?dl=1)**

In [13]:
import urllib

def download(url, filepath):
    response = urllib.request.urlretrieve(url, filepath)
    return response

In [14]:
prepared_data_link = 'https://www.dropbox.com/s/ygpbc5q0xtrsjqq/ProperSquatTechnique_cut.mp4?dl=1 '
dataset_zip = 'ProperSquatTechnique_cut.mp4'

download(prepared_data_link, dataset_zip)

('ProperSquatTechnique_cut.mp4', <http.client.HTTPMessage at 0x7b9427a42810>)

In [15]:
def inferenceOnVideo(videoPath):
    cap = cv2.VideoCapture(videoPath)
    cnt = 0
    n_frame = 2

    output_frames = []
    prev_val = -1
    squat_cnt = 0
    prev_squat_frame = 0
    process_start = time.time()

    while True:
        ret, im = cap.read()

        if not ret:
            break

        if cnt%n_frame == 0:
            output, angles_output = predict(im)
            temp_val = 0
            person_out = angles_output[0]

            if (int(person_out[0]) < 30) and (int(person_out[1]) < 30):
                if person_out[2] >= 0:
                    temp_val = 1

            if (prev_val == 0 and temp_val == 1) and (cnt - prev_squat_frame > 10):
                squat_cnt = squat_cnt + 1
                prev_squat_frame = cnt

            putTextOnImage(output, "Squat count: " + str(squat_cnt), 50, 50, (255,0,0))
            output_frames.append(output)

            prev_val = temp_val

        cnt = cnt + 1

    vid_write_start = time.time()
    print("total processing time", vid_write_start - process_start)
    height, width, _ = output_frames[0].shape
    size = (width,height)
    out = cv2.VideoWriter("out1.mp4",cv2.VideoWriter_fourcc(*'mp4v'), 10, size)

    for i in range(len(output_frames)):
        out.write(output_frames[i])

    print("video writing time", time.time() - vid_write_start)

    out.release()

In [19]:
start= time.time()
inferenceOnVideo("/content/ProperSquatTechnique_cut.mp4")
print(time.time() - start)

total processing time 41.98098635673523
video writing time 0.48085689544677734
42.469035387039185
